# Custom layers

We recommend using `tf.keras` as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.


In [3]:
import tensorflow as tf

In [4]:
print(tf.config.list_physical_devices('GPU'))

[]


## Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io) API in the tf.keras package, and the Keras layers are very useful when building your own models.


In [11]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer),
Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [13]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [14]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_7/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.41905028, -0.14225042, -0.54180515, -0.15413108,  0.56353897,
          0.00223833, -0.09507346, -0.296003  , -0.01942104, -0.34390724],
        [ 0.5273344 ,  0.34390193, -0.28642127, -0.5911583 , -0.16138011,
          0.22381091, -0.31642318,  0.03402799, -0.37771913, -0.3501223 ],
        [-0.43251273,  0.45806664,  0.38762993, -0.18399638,  0.13132632,
          0.41890818,  0.57048696,  0.14705741, -0.3811912 ,  0.37444776],
        [-0.09519547,  0.17809075,  0.29447842,  0.18742067, -0.58753335,
          0.6187343 ,  0.03235346,  0.0224219 ,  0.52709955,  0.30944228],
        [-0.14961857,  0.09334272,  0.41361946,  0.00412619, -0.60524535,
         -0.5115882 ,  0.2050752 , -0.14052328,  0.02605891, -0.6003347 ]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [15]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_7/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.41905028, -0.14225042, -0.54180515, -0.15413108,  0.56353897,
          0.00223833, -0.09507346, -0.296003  , -0.01942104, -0.34390724],
        [ 0.5273344 ,  0.34390193, -0.28642127, -0.5911583 , -0.16138011,
          0.22381091, -0.31642318,  0.03402799, -0.37771913, -0.3501223 ],
        [-0.43251273,  0.45806664,  0.38762993, -0.18399638,  0.13132632,
          0.41890818,  0.57048696,  0.14705741, -0.3811912 ,  0.37444776],
        [-0.09519547,  0.17809075,  0.29447842,  0.18742067, -0.58753335,
          0.6187343 ,  0.03235346,  0.0224219 ,  0.52709955,  0.30944228],
        [-0.14961857,  0.09334272,  0.41361946,  0.00412619, -0.60524535,
         -0.5115882 ,  0.2050752 , -0.14052328,  0.02605891, -0.6003347 ]],
       dtype=float32)>,
 <tf.Variable 'dense_7/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

1. `__init__` , where you can do all input-independent initialization
2. `build`, where you know the shapes of the input tensors and can do the rest of the initialization
3. `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in `build` is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [16]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [17]:
print([var.name for var in layer.trainable_variables])

['dense_7/kernel:0', 'dense_7/bias:0']


## Models: Composing layers

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.

Typically you inherit from `keras.Model` when you need the model methods like: `Model.fit`,`Model.evaluate`, and `Model.save` (see [Custom Keras layers and models](https://www.tensorflow.org/guide/keras/custom_layers_and_models) for details).

One other feature provided by `keras.Model` (instead of `keras.layers.Layer`) is that in addition to tracking variables, a `keras.Model` also tracks its internal layers, making them easier to inspect.

For example here is a ResNet block:

In [19]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [21]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [20]:
block.layers

In [22]:
len(block.variables)

18

In [23]:
block.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  4         
                                                                 
 batch_normalization (Batch  multiple                  4         
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           multiple                  4         
                                                                 
 batch_normalization_1 (Bat  multiple                  8         
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           multiple                  9         
                                                                 
 batch_normalization_2 (Bat  multiple                  12        


Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using `tf.keras.Sequential`:

In [24]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [25]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, None, None, 1)     4         
                                                                 
 batch_normalization_3 (Bat  (None, None, None, 1)     4         
 chNormalization)                                                
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 2)     4         
                                                                 
 batch_normalization_4 (Bat  (None, None, None, 2)     8         
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, None, None, 3)     9         
                                                                 
 batch_normalization_5 (Bat  (None, None, None, 3)     1